<a href="https://colab.research.google.com/github/fidanhsnva/SkillFactory/blob/main/Copy_%22DSF_ASOS_Build_and_Deploy_a_Recommender_in_3_Hours_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Festival x ASOS
## Build and Deploy a Recommender System in 3 Hours.

# Imports

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os

# Import training data

In [ ]:
train = pd.read_parquet("https://raw.githubusercontent.com/ASOS/dsf2020/main/dsf_asos_train_with_alphanumeric_dummy_ids.parquet")
valid = pd.read_parquet("https://raw.githubusercontent.com/ASOS/dsf2020/main/dsf_asos_valid_with_alphanumeric_dummy_ids.parquet")
dummy_users = pd.read_csv("https://raw.githubusercontent.com/ASOS/dsf2020/main/dsf_asos_dummy_users_with_alphanumeric_dummy_ids.csv", header=None).values.flatten().astype(str)
products = pd.read_csv("https://raw.githubusercontent.com/ASOS/dsf2020/main/dsf_asos_productIds.csv", header=None).values.flatten().astype(int)

In [ ]:
train.head()

,dummyUserId,productId
0,b'PIXcm7Ru5KmntCy0yA1K',10524048
1,b'd0RILFB1hUzNSINMY4Ow',9137713
2,b'Ebax7lyhnKRm4xeRlWW2',5808602
3,b'vtigDw2h2vxKt0sJpEeU',10548272
4,b'r4GfiEaUGxziyjX0PyU6',10988173


# The briefest intro to tf

Tensors

In [ ]:
x = tf.constant([1,2,3,4])
tf.math.square(x)

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([ 1,  4,  9, 16], dtype=int32)>

In [ ]:
tf.constant([[1,2,3],[4,5,6]], dtype = tf.float32)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [ ]:
x = tf.Variable([1,2,3,4], dtype = tf.float32)
x

<tf.Variable 'Variable:0' shape=(4,) dtype=float32, numpy=array([1., 2., 3., 4.], dtype=float32)>

Gradients

In [ ]:
with tf.GradientTape() as tape:
  y = tf.math.square(x)

In [ ]:
y

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 1.,  4.,  9., 16.], dtype=float32)>

In [ ]:
dy_dx = tape.gradient(y,x)
dy_dx

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([2., 4., 6., 8.], dtype=float32)>

Multiply and add tensors

In [ ]:
x = tf.constant([[1,2,3]], dtype=tf.float32)
Y = tf.constant([[1,2,3, 4], [1,2,3,4], [1,2,3,4]], dtype=tf.float32)

In [ ]:
tf.matmul(x,Y)

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[ 6., 12., 18., 24.]], dtype=float32)>

In [ ]:
z = tf.constant([10, 11, 12, 13], dtype=tf.float32)

In [ ]:
tf.matmul(x, Y) + z

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[16., 23., 30., 37.]], dtype=float32)>

This operation is very common in deep learning, so it has been abstracted:

In [ ]:
dl1 = tf.keras.layers.Dense(4, use_bias = True, weights = [Y,z])
dl1(x)

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[16., 23., 30., 37.]], dtype=float32)>

You can choose to apply a function to each value in the output

In [ ]:
dl2 = tf.keras.layers.Dense(4, use_bias = True, weights = [Y,z], activation = lambda x:x+1)
dl2(x)

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[17., 24., 31., 38.]], dtype=float32)>

We can put different layers together in a sequence:

In [ ]:
dl3 = tf.keras.layers.Dense(1, use_bias=False, \
                             weights=[tf.constant([[0], [1], [0], [1]], \
                                                  dtype=tf.float32)])

In [ ]:
x_b = dl2(x)
x_b

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[17., 24., 31., 38.]], dtype=float32)>

In [ ]:
dl3(x_b)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[62.]], dtype=float32)>

We can get more flexibility if you use tf.keras.model:

In [ ]:
class simple_model(tf.keras.Model):
  def __init__(self):
      super(simple_model,self).__init__()
      self.dl2 = tf.keras.layers.Dense(4, use_bias = True, weights = [Y,z], activation = lambda x:x+1)
      self.dl3 = tf.keras.layers.Dense(1, use_bias=False, \
                             weights=[tf.constant([[0], [1], [0], [1]], \
                                                  dtype=tf.float32)])
  def call(self,x):
    x_b = self.dl2(x)
    return self.dl3(x_b)

In [ ]:
sm =simple_model()
sm(x)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[62.]], dtype=float32)>

So far we have been setting the weights of the dense layers, but if we don't set the weights than weights get randomly chosen.

In [ ]:
dl6 = tf.keras.layers.Dense(4, use_bias=True)
dl6(x)

<tf.Tensor: shape=(1, 4), dtype=float32, numpy=array([[-3.0293803,  3.0663497,  2.1233385,  0.4467895]], dtype=float32)>

In [ ]:
dl6.get_weights()

[array([[-0.7639469 ,  0.2881403 , -0.9216829 ,  0.7368721 ],
        [-0.39119112,  0.33316886,  0.19552743,  0.60128915],
        [-0.49435034,  0.7039572 ,  0.8846555 , -0.49755362]],
       dtype=float32), array([0., 0., 0., 0.], dtype=float32)]

# Define a Recommender Model

The embedding layer gives a list of random numbers for each user and each product.

In [ ]:
embed1 = tf.keras.layers.Embedding(5,8)

In [ ]:
embed1(2)

<tf.Tensor: shape=(8,), dtype=float32, numpy=
array([ 0.03939882, -0.01750051,  0.01049685, -0.01101388, -0.04074287,
        0.02474223,  0.03345439, -0.02648175], dtype=float32)>

In [ ]:
embed1.get_weights()

[array([[ 0.04565668,  0.01802614,  0.00982577, -0.02069373,  0.01467462,
         -0.03788229, -0.00402718, -0.04378853],
        [ 0.02671686,  0.01625851, -0.03203268,  0.0487585 ,  0.02778449,
         -0.03032959,  0.0191387 , -0.00617224],
        [ 0.03939882, -0.01750051,  0.01049685, -0.01101388, -0.04074287,
          0.02474223,  0.03345439, -0.02648175],
        [ 0.02966264, -0.04565565, -0.0109917 ,  0.03751875,  0.01568342,
          0.04338393,  0.02583126, -0.02735187],
        [-0.03690867, -0.02768832, -0.00021479, -0.00683592, -0.0309788 ,
         -0.01464746,  0.04721781, -0.0290182 ]], dtype=float32)]

Scores can be found using the dot product.

In [ ]:
dummy_user_embedding = tf.keras.layers.Embedding(len(dummy_users), 6)
product_embedding = tf.keras.layers.Embedding(len(products), 6)

In [ ]:
dummy_user_embedding(1)

<tf.Tensor: shape=(6,), dtype=float32, numpy=
array([ 0.03028536,  0.0169034 , -0.01579468,  0.00061264, -0.00901758,
       -0.02777898], dtype=float32)>

In [ ]:
product_embedding(99)

<tf.Tensor: shape=(6,), dtype=float32, numpy=
array([-0.01741489,  0.01391072,  0.01982859, -0.00024699,  0.00224771,
       -0.00315019], dtype=float32)>

In [ ]:
tf.tensordot(dummy_user_embedding(1),product_embedding(99), axes = [[0],[0]])

<tf.Tensor: shape=(), dtype=float32, numpy=-0.0010469843>

We can score multiple products at the same time, which is what we need to create a ranking.

In [ ]:
example_product = tf.constant([1, 77, 104, 2062])
product_embedding(example_product)

<tf.Tensor: shape=(4, 6), dtype=float32, numpy=
array([[-0.02220537,  0.04136939,  0.00683858, -0.00405788, -0.04347848,
        -0.04487054],
       [-0.01646949, -0.02100272,  0.0151557 ,  0.01069765, -0.01973487,
        -0.02308404],
       [-0.04572801,  0.02932609, -0.0163287 ,  0.04251078,  0.03354562,
        -0.04800281],
       [-0.01771278, -0.02188824,  0.01729195,  0.04284317,  0.01883117,
        -0.03367139]], dtype=float32)>

In [ ]:
tf.tensordot(dummy_user_embedding(1),product_embedding(example_product),axes = [[0],[1]])

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([ 0.00064845,  0.00324576, -0.00116523, -0.00113344], dtype=float32)>

And we can score multiple users for multiple products which we will need to do if we are to train quickly.

But we need to map product ids to embedding ids.

In [ ]:
product_table = tf.lookup.StaticHashTable(
    tf.lookup.KeyValueTensorInitializer(tf.constant(products, dtype=tf.int32), 
                                        range(len(products))), -1)

Let's put those two things together

In [ ]:
class SimpleRecommender(tf.keras.Model):
    def __init__(self, dummy_users, products, length_of_embedding):
        super(SimpleRecommender, self).__init__()
        self.products = tf.constant(products, dtype=tf.int32)
        self.dummy_users = tf.constant(dummy_users, dtype=tf.string)
        self.dummy_user_table = tf.lookup.StaticHashTable(tf.lookup.KeyValueTensorInitializer(self.dummy_users, range(len(dummy_users))), -1)
        self.product_table = tf.lookup.StaticHashTable(tf.lookup.KeyValueTensorInitializer(self.products, range(len(products))), -1)
        
        self.user_embedding = tf.keras.layers.Embedding(len(dummy_users), length_of_embedding)
        self.product_embedding = tf.keras.layers.Embedding(len(products), length_of_embedding)

        self.dot = tf.keras.layers.Dot(axes = -1)
        
    def call(self, inputs):
        user = inputs[0]
        products = inputs[1]

        user_embedding_index = self.dummy_user_table.lookup(user)
        product_embedding_index = self.product_table.lookup(products)

        user_embedding_values = self.user_embedding(user_embedding_index)
        product_embedding_values = self.product_embedding(product_embedding_index)

        return tf.squeeze(self.dot([user_embedding_values, product_embedding_values]), 1)
    
    @tf.function
    def call_item_item(self, product):
        product_x = self.product_table.lookup(product)
        pe = tf.expand_dims(self.product_embedding(product_x), 0)
        
        all_pe = tf.expand_dims(self.product_embedding.embeddings, 0)#note this only works if the layer has been built!
        scores = tf.reshape(self.dot([pe, all_pe]), [-1])
        
        top_scores, top_indices = tf.math.top_k(scores, k=100)
        top_ids = tf.gather(self.products, top_indices)
        return top_ids, top_scores

In [ ]:
dummy_users

array(['pmfkU4BNZhmtLgJQwJ7x', 'UDRRwOlzlWVbu7H8YCCi',
       'QHGAef0TI6dhn0wTogvW', ..., 'lcORJ5hemOZc1iGo9z7k',
       '5CqDquDAszqJp27P7AL8', 'SSPNYxJMfuKhoe1dg24m'], dtype='<U20')

In [ ]:
products


array([ 8650774,  9306139,  9961521, ..., 12058614, 12058615, 11927550])

In [ ]:
sr1 = SimpleRecommender(dummy_users, products, 15)
sr1([tf.constant([['pmfkU4BNZhmtLgJQwJ7x'],['UDRRwOlzlWVbu7H8YCCi']]),tf.constant([[8650774,  9306139,  9961521],[12058614, 12058615, 11927550]])])

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 0.00107294, -0.00777534, -0.0041576 ],
       [-0.0047044 , -0.00320701,  0.00086349]], dtype=float32)>

# Creating a dataset

First create a tf.data.Dataset from the user purchase pairs.

In [ ]:
dummy_user_tensor = tf.constant(train[["dummyUserId"]].values, dtype=tf.string)
product_tensor = tf.constant(train[["productId"]].values, dtype=tf.int32)

dataset = tf.data.Dataset.from_tensor_slices((dummy_user_tensor, product_tensor))
for x, y in dataset:
    print(x)
    print(y)
    break

tf.Tensor([b'PIXcm7Ru5KmntCy0yA1K'], shape=(1,), dtype=string)
tf.Tensor([10524048], shape=(1,), dtype=int32)


In [ ]:
random_negatives_indexs = tf.random.uniform((7,), minval = 0, maxval = len(products), dtype = tf.int32)

In [ ]:
tf.gather(products, random_negatives_indexs)

<tf.Tensor: shape=(7,), dtype=int64, numpy=
array([ 8020122, 11590295, 11756713, 11533829, 10471910, 12273427,
       11842820])>

For each purchase let's sample a number of products that the user did not purchase. Then the model can score each of the products and we will know we are doing a good job if the product with the highest score is the product that the user actually purchased.

We can do this using dataset.map

In [ ]:
tf.one_hot(0,depth = 11)

<tf.Tensor: shape=(11,), dtype=float32, numpy=array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [ ]:
class Mapper():
    
    def __init__(self, possible_products, num_negative_products):
        self.num_possible_products = len(possible_products)
        self.possible_products_tensor = tf.constant(possible_products, dtype=tf.int32)
        
        self.num_negative_products = num_negative_products
        self.y = tf.one_hot(0, num_negative_products+1)
    
    def __call__(self, user, product):
        random_negatives_indexs = tf.random.uniform((self.num_negative_products,), minval = 0, maxval = self.num_possible_products, dtype = tf.int32)
        negatives = tf.gather(self.possible_products_tensor, random_negatives_indexs)
        candidates = tf.concat([product, negatives], axis=0)
        return (user, candidates), self.y

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((dummy_user_tensor, product_tensor)).map(Mapper(products, 10))
for (u, c),y in dataset:
  print(u)
  print(c)
  print(y)
  break

tf.Tensor([b'PIXcm7Ru5KmntCy0yA1K'], shape=(1,), dtype=string)
tf.Tensor(
[10524048  9197915 12944932 10715306 12319385  9527987 10537465 11553583
 10952762 13175230  9849906], shape=(11,), dtype=int32)
tf.Tensor([1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(11,), dtype=float32)


Let's bring the steps together to define a function which creates a dataset 

In [ ]:
def get_dataset(df, products, num_negative_products):
    dummy_user_tensor = tf.constant(df[["dummyUserId"]].values, dtype=tf.string)
    product_tensor = tf.constant(df[["productId"]].values, dtype=tf.int32)

    dataset = tf.data.Dataset.from_tensor_slices((dummy_user_tensor, product_tensor))
    dataset = dataset.map(Mapper(products, num_negative_products))
    dataset = dataset.batch(1024)
    return dataset

In [ ]:
for (u,c), y in get_dataset(train, products, 4):
  print(u)
  print(c)
  print(y)
  break

tf.Tensor(
[[b'PIXcm7Ru5KmntCy0yA1K']
 [b'd0RILFB1hUzNSINMY4Ow']
 [b'Ebax7lyhnKRm4xeRlWW2']
 ...
 [b'xuX9n8PHfSR0AP3UZ8ar']
 [b'iNnxsPFfOa9884fMjVPJ']
 [b'aD8Mn12im8lFPzXAY41P']], shape=(1024, 1), dtype=string)
tf.Tensor(
[[10524048 10818006  9286497 11937472 13146985]
 [ 9137713 11455407 10797275 12049325 10172148]
 [ 5808602  8782509  9430603 10524937  8259694]
 ...
 [11541336 11390873  9820527 10529177 10906660]
 [ 7779232 11938346 11923383 10728207 10309914]
 [ 4941259  9102997 11776806 10031596 11279983]], shape=(1024, 5), dtype=int32)
tf.Tensor(
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]], shape=(1024, 5), dtype=float32)


# Train a model

We need to compile a model, set the loss and create an evaluation metric. Then we need to train the model.

In [ ]:
model = SimpleRecommender(dummy_users, products, 15)
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True), optimizer = tf.keras.optimizers.SGD(learning_rate= 100.), metrics = tf.keras.metrics.CategoricalAccuracy())

model.fit(get_dataset(train, products, 100), validation_data = get_dataset(valid,products, 100), epochs = 5)

Epoch 1/5
162/162 [==============================] - 9s 49ms/step - loss: 4.6151 - categorical_accuracy: 0.0134 - val_loss: 4.6151 - val_categorical_accuracy: 0.0185
Epoch 2/5
162/162 [==============================] - 8s 49ms/step - loss: 4.6120 - categorical_accuracy: 0.0824 - val_loss: 4.6147 - val_categorical_accuracy: 0.0457
Epoch 3/5
162/162 [==============================] - 8s 49ms/step - loss: 4.5983 - categorical_accuracy: 0.1912 - val_loss: 4.6060 - val_categorical_accuracy: 0.0725
Epoch 4/5
162/162 [==============================] - 8s 49ms/step - loss: 4.5243 - categorical_accuracy: 0.2425 - val_loss: 4.5525 - val_categorical_accuracy: 0.0977
Epoch 5/5
162/162 [==============================] - 8s 49ms/step - loss: 4.3056 - categorical_accuracy: 0.2436 - val_loss: 4.4079 - val_categorical_accuracy: 0.1097


Let's do a manual check on whether the model is any good.

In [ ]:
test_product = 11698965

In [ ]:
print("Recs for item {}: {}".format(test_product, model.call_item_item(tf.constant(test_product, dtype=tf.int32))))

Recs for item 11698965: (<tf.Tensor: shape=(100,), dtype=int32, numpy=
array([11950358,  5801178, 10402703,  8661992, 11660467, 11888641,
       11499946, 12115410, 10505036,  7134751, 11374873, 10990591,
       11116710, 10636194, 12050426, 11448828,  9500372, 11230891,
       10370822,  7285091,  9109180,  5240244, 10375083, 11113894,
       11890074, 12261521, 11165000, 11491053, 10880400,  9923444,
       11856623, 11021553, 11541077, 10171156, 11378317, 10351406,
       10953790, 10487343,  9606234,  9412811, 10069701, 10394951,
       11387422, 13536031,  8713887,  8888567, 10636823, 10366899,
       10613554, 10214652, 11723487, 10965260,  9559937,  9426981,
       11701095, 10274163, 12241759, 10917206, 12537084,  9516257,
       11603928, 13329218,  8321496, 10677500, 10490457, 12606047,
       12064570, 12096403,  9200087, 10614054, 10636211, 10208752,
        8064059, 10943107, 11405773, 11301034, 12296543, 11393443,
       10605536, 12463221, 11409896, 11406238, 10976187, 1

# Save the model

In [ ]:
model_path = "models/recommender/1"

In [ ]:
inpute_signature = tf.TensorSpec(shape=(), dtype=tf.int32)

In [ ]:
signatures = { 'call_item_item': model.call_item_item.get_concrete_function(inpute_signature)}

In [ ]:
tf.saved_model.save(model, model_path, signatures = signatures)

INFO:tensorflow:Assets written to: models/recommender/1/assets


In [ ]:
imported_model = tf.saved_model.load('models/recommender/1')
list(imported_model.signatures.keys())

['call_item_item']

In [ ]:
result_tensor = imported_model.signatures['call_item_item'](tf.constant([14844847]))

from IPython.core.display import HTML

def path_to_image_html(path):
  return '<img src = https://images.asos-media.com/products/ugg-classic-mini-boots-in-black-suede/' + str(path) + '-2" width = "60" >'

result_df = pd.DataFrame(result_tensor['output_0'].numpy(), columns = ['ProductUrl']).head(5)
HTML(result_df.to_html(escape = False, formatters = dict(ProductUrl = path_to_image_html)))

,ProductUrl
0,
1,
2,
3,
4,


In [ ]:
imported_model.signatures['call_item_item'](tf.constant([14844847]))

In [ ]:
os.makedirs("dummy/0")
tf.saved_model.save(model, 'dummy/0')    
imported = tf.saved_model.load("dummy/0")
imported(tf.constant([14844847]))

In [ ]:
os.makedirs("dummy/1")
tf.saved_model.save(model, 'dummy/1',
                    model.call_item_item.get_concrete_function(tf.TensorSpec(shape=(), dtype=tf.int32)))      
list(imported_model.signatures.keys())

In [ ]:
imported_model.signatures['serving_default'](tf.constant([14844847]))

Zipping the saved model will make it easier to download.

In [ ]:
from zipfile import ZipFile
import os
# create a ZipFile object
with ZipFile('recommender.zip', 'w') as zipObj:
   # Iterate over all the files in directory
    for folderName, subfolders, filenames in os.walk("models"):
        for filename in filenames:
           #create complete filepath of file in directory
           filePath = os.path.join(folderName, filename)
           # Add file to zip
           zipObj.write(filePath)